In [1]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langgraph_supervisor import  create_supervisor
from langgraph.prebuilt import  create_react_agent
from openai import AzureOpenAI
import os
import azure.identity
from azure.identity import ClientSecretCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

In [2]:
import langgraph

In [3]:
try:
    keyVaultName = os.environ["KEY_VAULT_NAME"]
except KeyError:
    # Get input from user if not set
    keyVaultName = input("Please enter your Key Vault name: ")
    # Save for future cells in this session
    os.environ["KEY_VAULT_NAME"] = keyVaultName

In [4]:
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

In [5]:
azure_openai_endpoint=client.get_secret(name="aoai-endpoint").value
azure_openai_api_key=client.get_secret(name="aoai-api-key").value
azure_openai_api_version = "2024-02-15-preview"

In [7]:
model = AzureChatOpenAI(model="gpt-4o", api_key=azure_openai_api_key, api_version=azure_openai_api_version, azure_endpoint=azure_openai_endpoint)

In [8]:
# Create specialized agents

def add_floats(a: float, b:float) -> float:
    """Add two floats."
    """""
    return a + b


def multiply_floats(a: float, b: float) -> float:
    """Multiply two floats.
    """
    return a * b

def web_search(query:str) -> str:
    """Search the web for information.
    """
    return (
        "Here are the headcounts for each of the FAANG companies in 2025:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=model,
    tools=[add_floats, multiply_floats],
    name="math_expert",
    prompt="You are a math agent. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)


In [9]:
#Create supervisor workflow
workflow = create_supervisor(
    agents=[math_agent, research_agent],
    model=model,
    prompt=("You are a team supervisor managing a research expert and a math expert."
        "For current events, use research_agent."
        "For math problems, use math_agent.")
)

In [10]:
# Compile and run the workflow

app = workflow.compile()
result = app.invoke(
    {
        "messages":[
            {
                "role": "user",
                "content": "what's the combined headcount of the FAANG companies in 2024?"
            }
        ]
    }
)


In [11]:
print(f"The agent returned :{result['messages'][-1].content}")

The agent returned :The 2024 headcount data for each FAANG company is not readily available, but as of 2025, the combined headcount is as follows:

1. Meta (Facebook): 67,317 employees
2. Apple: 164,000 employees
3. Amazon: 1,551,000 employees
4. Netflix: 14,000 employees
5. Alphabet (Google): 181,269 employees

These numbers total up to a combined headcount of 1,977,586 employees. Keep in mind that these figures are for 2025, and actual 2024 numbers may vary.


In [12]:
# Show agent's step by step response
for message in result["messages"]:
    print(message.pretty_print())

================================ Human Message =================================

what's the combined headcount of the FAANG companies in 2024?
None
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_Xw0ZseuZBMjTpt8rxnkobiIU)
 Call ID: call_Xw0ZseuZBMjTpt8rxnkobiIU
  Args:
None
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
None
================================== Ai Message ==================================
Name: research_expert

I couldn't find specific 2024 headcount data for each of the FAANG companies. However, here are the headcounts for each of the FAANG companies in 2025:

1. **Facebook (Meta)**: 67,317 employees.
2. **Apple**: 164,000 employees.
3. **Amazon**: 1,551,000 employees.
4. **Netflix**: 14,000 employees.
5. **Google (Alphabet)**: 181,269 employees.

Please note t

In [4]:
import langgraph.checkpoint.memory
import langgraph.store
import langgraph.store.base

import langgraph.checkpoint


dir(langgraph.checkpoint)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base',
 'memory',
 'serde']